In [24]:
import math
import string

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset, IterableDataset

import torchdata
import torchtext

import re
import glob
import warnings
import unicodedata
import string

torch.device('cuda')

device(type='cuda')

In [25]:
def get_files(path):
    return glob.glob(path)

def unicodeToAscii(word):
    return ''.join(c for c in unicodedata.normalize('NFD', word)
        if unicodedata.category(c) != 'Mn' and c in all_letters
    )

def readLine(fname):
    file_name = re.findall(r'/data/names/(.*).txt', fname)[0]
    with open(fname,encoding='utf-8') as file:
        return file_name,[unicodeToAscii(line.strip()) for line in file]
    
def getCategory(path):
    files = get_files(path)
    category = [re.findall(r'/data/names/(.*).txt', file_name)[0] for file_name in files]
    return category

def getCatToName(path):
    files = get_files(path)
    cats = []
    catToName = {}
    for file in files:
        cat, name = readLine(file)
        cats.append(cat)
        catToName[cat] = name
    return cats, catToName

In [26]:
all_letters = string.ascii_letters + ".,';-"
n_letters = len(all_letters) + 1 #for EOS

In [27]:
### Label
path = './data/names/*.txt'
### Lang to Name
all_cat, catToLine = getCatToName(path)

In [28]:
class RNN_GEN_NAME(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_categories):
        super(RNN_GEN_NAME, self).__init__()
        
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.output_size = output_size
        self.n_categories = n_categories
        
        self.inputToHidden = nn.Linear(input_size+hidden_size+n_categories, hidden_size)
        self.inputToOutput = nn.Linear(input_size+hidden_size+n_categories, output_size)
        self.outToOut = nn.Linear(hidden_size+output_size, output_size)
        
        self.dropOut = nn.Dropout(0.3)
        self.act1 = nn.LeakyReLU()
        self.act2 = nn.LeakyReLU()
        self.softMax = nn.LogSoftmax()
        
    def forward(self, category, inp, hid):
        input_comb = torch.cat((category,inp,hid), 1)
        
        hidden = self.inputToHidden(input_comb)
        output = self.inputToOutput(input_comb)
        
        output_combined = torch.cat((hidden, output), 1)
        
        output = self.outToOut(output_combined)
        output = self.dropOut(output)
        output = self.softMax(output)
        
        return output, hidden
    
    def initHidden(self):
        return torch.zeros((1, self.hidden_size))

In [29]:
import random, math

def randomChoice(listToChoice):
    return listToChoice[random.randint(0, len(listToChoice)-1)]

def randomPair(catList, catToName):
    randomCat = randomChoice(catList)
    lineList = catToName[randomCat]
    randomLine = randomChoice(lineList)
    return randomCat, randomLine

def randomExample(catList, oheCat, catToName):
    cat, line = randomPair(catList, catToName)
    catTensor = oheCat[cat]
    lineTensor = lineToTensor(line)
    return cat, line, catTensor, lineTensor

In [30]:
def getIndex(listToSearch, toSearch):
    index = math.inf
    lenList = len(listToSearch)

    for i in range(lenList):
        if listToSearch[i] == toSearch:
            index = i
            break

    if index >= 0:
        return index

def lineToTensor(line, letters=all_letters):
    tensor = torch.zeros([len(line), len(letters)])
    for i, l in enumerate(line):
        l_to_index = getIndex(letters, l)
        tensor[i][l_to_index] = 1
    return tensor

def targetLineTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

In [31]:
ex = lineToTensor('abd')
print(torch.unsqueeze(ex, 1).size())
ex = targetLineTensor('abd')

torch.Size([3, 1, 57])


In [32]:
def ohe_cat(cat):
    lenCat = len(cat)
    tensor = torch.zeros([lenCat, lenCat])
    catToTensor = {}
    for i in range(lenCat):
        tensor[i][i] = 1
        catToTensor[cat[i]] = tensor[i]
    return catToTensor, tensor
    
def getIndex(listToSearch, toSearch):
    index = math.inf
    lenList = len(listToSearch)
    
    for i in range(lenList):
        if listToSearch[i] == toSearch:
            index = i
            break
    
    if index >= 0:
        return index 

In [33]:
ohe_cat_mapped_tensor, oheCatTensor = ohe_cat(all_cat)

In [34]:
import time
import math

rnn = RNN_GEN_NAME(n_letters, 512, n_letters, len(all_cat))
lossFunc = nn.NLLLoss()
lr = 0.05

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def train(category_tensor, line_tensor, target_tensor):
    target_tensor = torch.unsqueeze(target_tensor, -1)
    line_tensor = torch.unsqueeze(line_tensor, 1)
    
    hidden = rnn.initHidden()
    rnn.zero_grad()
    loss = torch.Tensor([0])
    
    output = None
    
    for i in range(line_tensor.shape[0]):
        print(category_tensor.shape, line_tensor.shape, hidden.shape)
        output, hidden = rnn(category_tensor, line_tensor, hidden)
        l = lossFunc(output, target_tensor[i])
        loss += l
        
    loss.backward()

    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-lr)

    return output, loss.item() / line_tensor.size(0)
    

In [35]:
n_iters = 100000
print_every = 1000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every ``plot_every`` ``iters``

start = time.time()

for iter in range(1, n_iters + 1):
    cat, line, cat_tensor, line_tensor = randomExample(all_cat, ohe_cat_mapped_tensor, catToLine)
    target_tensor = targetLineTensor(line)
    output, loss = train(cat_tensor, line_tensor, target_tensor)
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

torch.Size([18]) torch.Size([3, 1, 57]) torch.Size([1, 512])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
from io import open
import glob
import os
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

def findFiles(path): return glob.glob(path)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    with open(filename, encoding='utf-8') as some_file:
        return [unicodeToAscii(line.strip()) for line in some_file]

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
all_categories = []
for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

if n_categories == 0:
    raise RuntimeError('Data not found. Make sure that you downloaded data '
                       'from https://download.pytorch.org/tutorial/data.zip and extract it to '
                       'the current directory.')

print('# categories:', n_categories, all_categories)
print(unicodeToAscii("O'Néàl"))

In [ ]:
import torch
import torch.nn as nn
torch.device('cuda')

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        with torch.device('cuda'):
            super(RNN, self).__init__()
            self.hidden_size = hidden_size
    
            self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
            self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
            self.o2o = nn.Linear(hidden_size + output_size, output_size)
            self.dropout = nn.Dropout(0.1)
            self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        with torch.device('cuda'):
            input_combined = torch.cat((category, input, hidden), 1)
            hidden = self.i2h(input_combined)
            output = self.i2o(input_combined)
            output_combined = torch.cat((hidden, output), 1)
            output = self.o2o(output_combined)
            output = self.dropout(output)
            output = self.softmax(output)
            return output, hidden

    def initHidden(self):
        with torch.device('cuda'):
            return torch.zeros(1, self.hidden_size)

In [ ]:
# One-hot vector for category
def categoryTensor(category):
    with torch.device('cuda'):
        li = all_categories.index(category)
        tensor = torch.zeros(1, n_categories, device='cuda')
        tensor[0][li] = 1
        return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    with torch.device('cuda'):
        tensor = torch.zeros(len(line), 1, n_letters, device='cuda')
        for li in range(len(line)):
            letter = line[li]
            tensor[li][0][all_letters.find(letter)] = 1
        return tensor

# ``LongTensor`` of second letter to end (EOS) for target
def targetTensor(line):
    with torch.device('cuda'):
        letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
        letter_indexes.append(n_letters - 1) # EOS
        return torch.LongTensor(letter_indexes).to('cuda')

In [ ]:
# Make category, input, and target tensors from a random category, line pair
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    with torch.device('cuda'):
        category = randomChoice(all_categories)
        line = randomChoice(category_lines[category])
        return category, line

def randomTrainingExample():
    with torch.device('cuda'):
        category, line = randomTrainingPair()
        category_tensor = categoryTensor(category)
        input_line_tensor = inputTensor(line)
        target_line_tensor = targetTensor(line)
        return category_tensor, input_line_tensor, target_line_tensor

In [ ]:
criterion = nn.NLLLoss().to('cuda')

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    with torch.device('cuda'):
        target_line_tensor.unsqueeze_(-1).to('cuda')
        hidden = rnn.initHidden()
    
        rnn.zero_grad()
    
        loss = torch.Tensor([0]).to('cuda') # you can also just simply use ``loss = 0``
    
        for i in range(input_line_tensor.size(0)):
            output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
            l = criterion(output, target_line_tensor[i]).to('cuda')
            loss += l
    
        loss.backward()
    
        for p in rnn.parameters():
            p.data.add_(p.grad.data, alpha=-learning_rate)
    
        return output, loss.item() / input_line_tensor.size(0)

In [ ]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
rnn = RNN(n_letters, 128, n_letters).to('cuda')

n_iters = 100000
print_every = 500
plot_every = 500
all_losses = []
total_loss = 0 # Reset every ``plot_every`` ``iters``

start = time.time()

for iter in range(1, n_iters + 1):
    with torch.device('cuda'):
        output, loss = train(*randomTrainingExample())
        total_loss += loss
    
        if iter % print_every == 0:
            print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))
    
        if iter % plot_every == 0:
            all_losses.append(total_loss / plot_every)
            total_loss = 0

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(all_losses)


In [ ]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

In [38]:
samples('Arabic', 'QSA')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument tensors in method wrapper_CUDA_cat)

In [37]:
catToLine['Arabic']

['Khoury',
 'Nahas',
 'Daher',
 'Gerges',
 'Nazari',
 'Maalouf',
 'Gerges',
 'Naifeh',
 'Guirguis',
 'Baba',
 'Sabbagh',
 'Attia',
 'Tahan',
 'Haddad',
 'Aswad',
 'Najjar',
 'Dagher',
 'Maloof',
 'Isa',
 'Asghar',
 'Nader',
 'Gaber',
 'Abboud',
 'Maalouf',
 'Zogby',
 'Srour',
 'Bahar',
 'Mustafa',
 'Hanania',
 'Daher',
 'Tuma',
 'Nahas',
 'Saliba',
 'Shamoon',
 'Handal',
 'Baba',
 'Amari',
 'Bahar',
 'Atiyeh',
 'Said',
 'Khouri',
 'Tahan',
 'Baba',
 'Mustafa',
 'Guirguis',
 'Sleiman',
 'Seif',
 'Dagher',
 'Bahar',
 'Gaber',
 'Harb',
 'Seif',
 'Asker',
 'Nader',
 'Antar',
 'Awad',
 'Srour',
 'Shadid',
 'Hajjar',
 'Hanania',
 'Kalb',
 'Shadid',
 'Bazzi',
 'Mustafa',
 'Masih',
 'Ghanem',
 'Haddad',
 'Isa',
 'Antoun',
 'Sarraf',
 'Sleiman',
 'Dagher',
 'Najjar',
 'Malouf',
 'Nahas',
 'Naser',
 'Saliba',
 'Shamon',
 'Malouf',
 'Kalb',
 'Daher',
 'Maalouf',
 'Wasem',
 'Kanaan',
 'Naifeh',
 'Boutros',
 'Moghadam',
 'Masih',
 'Sleiman',
 'Aswad',
 'Cham',
 'Assaf',
 'Quraishi',
 'Shalhoub',
 '